# Multi-Class Classification - Language Classification

This notebook implements the method presented in Goldberg's [2017] book "Neural Network Methods for Natural Language Processing". It shows the steps you need to go through in order to successfully train a classifier, and it should also, so I hope, illustrate the notational differences between Goldberg and standard machine learning literature.

$NOTE$: There is no cross-validation etc. to find optimal parameters. This is simply to show how multi-class classification works. This will be part of a tutorial session and all other concepts will be explained there.

Author: Phillip Ströbel

## Getting and cleaning the data

The data consists of downloaded Wikipedia articles (see `urls.txt`) in German, English, French, Spanish, Italian and Finnish (instead of "O" in Goldberg). The data is in HTML, so we need to some preprocessing to get the text out of it. We also restrict ourselfes to the characters from a to z in the alphabet (as described in Goldberg). In this fashion, we get rid of all the Umlauts (ä, ö, ü) and all other characters with diacritics (as, e.g., the é or ç in French). Note however, that if these characters ocurring in bigrams would probably be good features. In some way, we still keep the information "special character" by not fully deleting the character, but by replacing it by the dollar sign "\$". Furthermore, we replace all punctuation marks and digits by dollar signs as well. As such, all special characters, digits, and punctuation marks are mapped to $. The space will be replaced by an underscore "\_". We then represent each langauge by 28 characters, as is suggested by Goldberg.

### Cleaning HTML
We first strip the HTML to get only the text of the Wikipedia page.

#### Get the html files

In [1]:
import re
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
from collections import defaultdict

seed = np.random.seed(seed=200)  # set a seed for random, so results are reproducible

article_dict = defaultdict(lambda: defaultdict(str))

regex = r'[\n ]{2,}'
pattern = re.compile(regex)

urls = open('urls.txt', 'r').readlines()

for url_index, url in enumerate(urls):
    language = url[8:10]
    doc_id = 'doc_%d' % url_index
    html = urlopen(url.strip()).read()    
    soup = BeautifulSoup(html, 'html.parser')
    raw = soup.body.get_text()  # only get text from the text body (this excludes headers and should exclude navigation bars)
    raw = re.sub(pattern, ' ', raw)  # replace multiple breaks and spaces by only one space
    raw = re.sub(r'\n', ' ', raw)  # replace every line break with a space
    article_dict[language][doc_id] = raw.lower()  # assign each text to its language and lower all uppercase characters

### Preprocessing --> prepare the text
replace special characters and digits

In [2]:
preprocessed_dict = defaultdict(lambda: defaultdict(str))

abc = r'[a-z]'
abc_pattern = re.compile(abc)

for lang, doc in article_dict.items():
    for doc, text in doc.items():
        for char in text:
            if re.match(abc_pattern, char):
                preprocessed_dict[lang][doc] += char
            elif re.match(' ', char):
                preprocessed_dict[lang][doc] += '_'
            else:
                preprocessed_dict[lang][doc] += '$'

### Count bigrams --> Feature extraction

The distribution of bigrams will be our only feature. We could extend this by taking into account other n-grams.

In [3]:
charset = 'abcdefghijklmnopqrstuvwxyz$_'  # define the character set we want to use

In [6]:
from itertools import combinations_with_replacement, permutations

def bigrams(text):
    """
    Function to extract bigrams from text and calculate their distribution
    :param text: text string
    :return: dictionary containing bigrams as keys, and the normalised count as values
    """
    combs = combinations_with_replacement(charset, 2)
    perms = permutations(charset, 2)
    bigram_dict = dict()
    
    for comb in set(list(combs) + list(perms)):
        bigram_dict[''.join(comb)] = 0
        
    doc_length = len(text)
    
    for i in range(0, len(text)-1):
        bigram = text[i] + text[i+1]
        bigram_dict[bigram] += 1
                
    for bigram, count in bigram_dict.items():
        bigram_dict[bigram] = count/doc_length

    return bigram_dict              

### Put data into pandas dataframe
The pandas dataframe allows us to conveniently represent all the data we need in one table. So let's do this. But first we need to extract the features.

In [7]:
bigram_dict_full = defaultdict(lambda: defaultdict(dict))

for lang, doc in preprocessed_dict.items():
    for doc, text in sorted(doc.items()):
        bigram_dict = bigrams(text)
        bigram_dict_full[lang][doc] = bigram_dict

In [8]:
import pandas as pd

col_names = ['y'] + sorted(bigram_dict_full['en']['doc_0'].keys())
my_df = dict()

for col in col_names:
    my_df[col] = list()
    
df = pd.DataFrame(my_df)

for lang, doc in bigram_dict_full.items():
    for key, value in doc.items():
        df_obj = value
        df_obj['y'] = lang
        df = df.append(df_obj, ignore_index=True)
        
df.head()
        

,$$,$_,$a,$b,$c,$d,$e,$f,$g,$h,...,zq,zr,zs,zt,zu,zv,zw,zx,zy,zz
0,0.063981,0.030567,0.000434,0.000346,0.000499,0.000193,0.000346,0.000201,0.000177,0.000290,...,0.000000,0.000000,0.000000,0.000000,0.000008,0.0,0.000000,0.0,0.000000,0.000016
1,0.071807,0.033520,0.000514,0.001359,0.000879,0.000597,0.000821,0.000390,0.000406,0.000373,...,0.000000,0.000000,0.000025,0.000008,0.000050,0.0,0.000041,0.0,0.000000,0.000017
2,0.052874,0.028307,0.000814,0.000423,0.000462,0.000282,0.000454,0.000243,0.000172,0.000227,...,0.000000,0.000000,0.000039,0.000000,0.000016,0.0,0.000000,0.0,0.000016,0.000008
3,0.075421,0.035189,0.000400,0.000466,0.000562,0.000268,0.000303,0.000224,0.000308,0.000189,...,0.000000,0.000004,0.000000,0.000000,0.000013,0.0,0.000000,0.0,0.000009,0.000000
4,0.062161,0.028908,0.000513,0.000328,0.000520,0.000356,0.000629,0.000192,0.000185,0.000171,...,0.000007,0.000007,0.000000,0.000000,0.000027,0.0,0.000000,0.0,0.000014,0.000021


In [9]:
df.shape

(60, 785)

Now we split the data into the label vector \begin{equation}\mathbf{y}\end{equation} and a training data matrix \begin{equation}\mathbf{X}\end{equation}. But first, we shuffle the df and split it into a training and a test set.

Split the data into a train set and a test set

In [10]:
train = df.sample(frac=0.9, random_state=seed)
test = df.drop(train.index)

Define the different sets

In [11]:
# for training
y_train = train.y
X_train = train.drop('y', axis=1)

# for testing
y_test = test.y
X_test = test.drop('y', axis=1)

Check the shapes

In [12]:
print('Training samples shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test samples shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training samples shape:  (54, 784)
Training labels shape:  (54,)
Test samples shape:  (6, 784)
Test labels shape:  (6,)


Moreover, it is necessary for many machine learning tasks to standardise the data. Our aim is for each feature to be represented by a column vector in which values have zero mean and unit variance.

In [32]:
def normalise_matrix(matrix, mean_and_std=None):
    """
    normalises the data matrix (normalise each datapoint to zero mean and unit variance.)
    :param matrix: input matrix
    :param mean_and_std: provide mean and std as tuples in list for normalisation of test data
    :return: normalised matrix and list consisting of tuples containing mean and std
    """
    normalised = np.ones(matrix.shape)
    
    if mean_and_std == None:
        
        mean_std_list = list()

        for col_index, col in enumerate(matrix):
            mean = matrix[col].mean()
            std = matrix[col].std()
            mean_std_list.append((mean, std))
            for row_index, item in enumerate(matrix[col]):
                try:
                    normalised[row_index, col_index] = (item - mean)/std
                except ZeroDivisionError:
                    normalised[row_index, col_index] = 0.0
        return normalised, mean_std_list
    else:
        for col_index, col in enumerate(matrix):
            for row_index, item in enumerate(matrix[col]):
                try:
                    mean = mean_and_std[col_index][0]
                    std = mean_and_std[col_index][1]
                    normalised[row_index, col_index] = (item - mean)/std
                except ZeroDivisionError:
                    normalised[row_index, col_index] = 0.0
        return normalised

normalise the train and test set (use values from training set for standardisation of test set). in sklearn, there are simple methods which help you with this!

In [51]:
train_norm, train_mean_std = normalise_matrix(X_train)
test_norm = normalise_matrix(X_test, train_mean_std)

We should binarise our labels, although libraries like sklearn can also deal with non-numeric data

In [38]:
from sklearn import preprocessing

lb = preprocessing.LabelBinarizer()
lb.fit(['en', 'fr', 'de', 'it', 'es', 'fi'])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [39]:
lb.classes_

array(['de', 'en', 'es', 'fi', 'fr', 'it'], dtype='<U2')

We do this for both our training and test labels:

In [40]:
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

Labels are now one-hot encoded:

In [41]:
y_train[0]

array([0, 0, 0, 1, 0, 0])

We almost have everything now. However, we need to take care of the bias and the weight matrix. The hypothesis ŷ is given by:
\begin{equation}
\mathbf{\hat{y}}=\mathbf{x}\cdot\mathbf{W}+\mathbf{b}
\end{equation}
We can achieve this by appending 1 to each feature vector x, and the whole weight vector b to the weight matrix W. This is called the bias trick. Note that the dimensions of X_train change, and that the weight matrix W will have match the dimensions (same number of rows as X has columns).

In [57]:
bias_vector = np.ones([train_norm.shape[0], 1])
X_train = np.append(train_norm, bias_vector, axis=1)
X_train.shape

(54, 785)

In [58]:
# initialise weight matrix with small weights

np.random.seed(seed=200)

W = np.random.randn(X_train.shape[1], len(lb.classes_)) * 0.0001
#W = np.zeros([X.shape[1], len(lb.classes_)])

We see that the dimensions are right. The dot product of a specific row from X_train and the weight matrix W constitutes a forward pass and calculates the score for each class.

In [59]:
W.shape

(785, 6)

In [60]:
X_train.shape

(54, 785)

In [64]:
X_train[5:6].dot(W)

array([[-0.0008988 , -0.00167437,  0.00149301,  0.00149552,  0.0036825 ,
         0.00016393]])

We see that the values for the highest score of the dot product is not the score of the true label. Our aim is to change this by implementing a support vector classifier.

In [65]:
X_train[5:6].dot(W).max(axis=1)

array([0.0036825])

In [68]:
X_train[5:6].dot(W)[0][y_train[5:6].argmax()]

-0.0016743673923002834

Important: we follow kind of a naive implementation. The aim is to be able to understand what is going on!

In order to quantify how good (or how bad) our weight matrix W can predict the data in our training set, we need to implement a loss function. Here we take a go at the hinge loss, which tries to predict the correct class with a margin of at least one to all other classes (or in this case, like presented in Goldberg, to the class which does not equal the true class, but which scores highest). In my understanding, this is a one-vs-one approach (true class vs. class with highest score (but doesn't equal the true class)).

In [116]:
def hinge_loss(x, y, W, index):
    """
    Calculates the loss of a single data point by taking the prediction of the correct value and the the prediction of
    the value of next highest score, following Crammer and Singer (2001)
    :param x: sample point x as a vector
    :param y: correct label y for x as a vector
    :param W: weight matrix
    :param index: column index of data matrix X
    :return: loss
    """
    loss = 0
    y_index = y[index].argmax()
    y_value = x.dot(W)[y_index]
    y_hat_max_value = np.delete(x.dot(W), y_index).max()
    #for j in range(0, y.shape[1]):  # in case we wanted to classify against all other classes (one-vs-all) --> currently one-vs-one
        #if j == y_index:
            #continue
    loss += max(0, 1 - (y_value - y_hat_max_value))
    return loss

With matrix multiplication, we could get all the scores at once. In the following, however, we focus on an approach which takes sample by sample and calculates the loss and the gradients.

In [69]:
scores = X_train.dot(W)  # simple matrix multiplication to get all scores

In [70]:
scores

array([[-7.36822338e-03, -5.38690529e-03, -4.39624523e-03,
        -6.44164976e-03,  9.18749565e-04,  3.58689231e-03],
       [-2.99758798e-03,  1.00016692e-03,  3.83380346e-05,
        -8.53791583e-04,  3.83284607e-03,  1.52095946e-04],
       [-1.60844274e-04,  5.80906353e-03,  2.94484507e-03,
         1.73589374e-03,  6.13581561e-04,  3.34736463e-03],
       [-9.93770414e-04, -1.85561820e-04, -5.63257159e-04,
         4.29132292e-03, -1.78035786e-03, -2.21281342e-03],
       [-1.95033302e-03,  7.82514989e-05,  1.51759145e-03,
        -8.38414430e-04, -1.77793019e-03, -4.54139891e-03],
       [-8.98801826e-04, -1.67436739e-03,  1.49301055e-03,
         1.49552287e-03,  3.68249626e-03,  1.63930726e-04],
       [ 3.93043664e-04,  2.60245967e-03,  7.27111868e-04,
         5.23099807e-03, -1.42932688e-03, -1.65149922e-03],
       [ 9.09820602e-04,  1.28445802e-03,  6.74879950e-04,
         2.00379697e-03,  7.29421319e-04, -3.86434529e-03],
       [-5.95409881e-03, -1.29857122e-03,  1.372

In [74]:
def gradient(X, y, W):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :return: loss and Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    total_loss = 0.0
    
    for index, x in enumerate(X):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value))
        total_loss += loss
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        if loss > 0:  # not sure whether we need this if statement
            dW[:, y_hat_max_index] += x.transpose()
            dW[:, y_index] -= x.transpose()
            
    return total_loss, dW
    

In [75]:
def gradient_descent(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        loss, dW = gradient(X, y, W_learned)
        print(loss)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [76]:
W_star = gradient_descent(X_train, y_train, W, eta=0.001, steps=10)

54.20764584833195
17.178159125332577
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


### Testing
Let's test if our learned representation of the data is any good at classifying the data in the test set. Of course we need the bias in our test set as well!

In [77]:
bias_vector_test = np.ones([X_test.shape[0], 1])
X_test = np.append(test_norm, bias_vector_test, axis=1)

In [79]:
for index, x in enumerate(X_test.dot(W_star)):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

1 1
3 3
3 3
0 0
4 4
4 4


Not too bad! But Goldberg mentioned something about regularisation, so we should take this into account as well!

In [97]:
def gradient_reg(X, y, W, lam):
    """
    compute the gradient
    :param X: data matrix (train) 
    :param y: the corresponding 
    :param W: weight matrix
    :param lam: reguliser lambda
    :return: Jacobian dW with all gradients
    """
    dW = np.zeros(W.shape)
    
    total_loss = 0.0
    
    for index, x in enumerate(X):
        y_index = y[index].argmax()
        y_value = x.dot(W)[y_index]
        y_hat_max_value = np.delete(x.dot(W), y_index).max()
        loss = max(0, 1 - (y_value - y_hat_max_value)) + lam * np.linalg.norm(W, 2)
        total_loss += loss
        y_hat_max_index = np.delete(x.dot(W), y_index).argmax() + 1
        if loss > 0:  # not sure whether we need this if statement
            dW[:, y_hat_max_index] += x.transpose()
            dW[:, y_index] -= x.transpose()
        
    dW += lam * W
            
    return total_loss, dW

def gradient_descent_reg(X, y, W, eta, steps):
    """
    Perform gradient descent for a number of times with a fixed learning rate eta
    :param X: data matrix
    :param y: labels
    :param W: weight matrix
    :param eta: learning rate
    :param steps: number of times gradient descent should be performed
    :return: learned representation matrix W_learned
    """
    W_learned = W.copy()
    
    for step in range(0, steps):
        loss, dW = gradient_reg(X, y, W_learned, -2)
        print(loss)
        W_learned = W_learned - eta * dW
        
    return W_learned
    

In [98]:
W_star_reg = gradient_descent_reg(X_train, y_train, W, eta=0.001, steps=10)

53.8924078298456
-19.915693715527116
-34.72686132199791
-34.796315044641915
-34.86590767473121
-34.93563949008068
-35.005510769060876
-35.0755217905989
-35.145672834180196
-35.21596417984851


In [99]:
for index, x in enumerate(X_test.dot(W_star_reg)):
    pred = x.argmax()
    true_label = y_test[index].argmax()
    print(pred, true_label)

1 1
3 3
3 3
0 0
4 4
4 4


If we look at the two different weight matrices (one regularised, the other not), we notice the following:

In [100]:
W_star[0:5]

array([[-0.00204807,  0.00598506, -0.00640592,  0.0087149 , -0.00527147,
        -0.00084924],
       [-0.00441286,  0.00444758, -0.01524402, -0.00293755,  0.01245237,
         0.00586772],
       [-0.00745913, -0.00624353,  0.01561007, -0.00880273,  0.01712681,
        -0.00989171],
       [ 0.01586253, -0.00306909, -0.00304522, -0.00704329,  0.00327248,
        -0.00608096],
       [-0.00093228, -0.00380838,  0.0025021 , -0.00890512,  0.02599701,
        -0.01478257]])

In [101]:
W_star_reg[0:5]

array([[-0.00208552,  0.00609405, -0.00695889,  0.00887298, -0.0052297 ,
        -0.00056512],
       [-0.00449297,  0.00452838, -0.01590587, -0.00299069,  0.00967119,
         0.00936672],
       [-0.00759443, -0.00635661,  0.01505479, -0.00896256,  0.01282986,
        -0.00462442],
       [ 0.01615017, -0.00312473, -0.00260994, -0.00717084,  0.00172759,
        -0.00507789],
       [-0.00094926, -0.00387774,  0.00249645, -0.00906628,  0.01961233,
        -0.00814331]])

## By the way ...
### In scikit-learn it's much easier to implement this :-)


In [88]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, multi_class='crammer_singer', loss='hinge')
clf.fit(X_train, train.y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=0,
     tol=0.0001, verbose=0)

In [89]:
clf.predict(X_test)

array(['en', 'fi', 'fi', 'de', 'fr', 'fr'], dtype=object)

In [90]:
test.y

4     en
12    fi
16    fi
26    de
41    fr
42    fr
Name: y, dtype: object

We see that with our naive implementation, we do not much worse than with scikit's. scikit's implementation is of course much more elaborate and uses the vectorised operation and possibly other optimisation techniques in order to make its SVM (or SVC) better.